In [1]:
import numpy as np
import random
from helpers import *
from costs import *
from gradient_descent import *
from stochastic_gradient_descent import *

random.seed(10)

In [2]:
y, x, ids = load_csv_data(data_path="datas/train.csv", sub_sample=False)
x.shape

(250000, 30)

In [3]:
w = np.random.rand(x.shape[1])
compute_loss(y, x, w)

4262084.377439185

In [4]:
gradient = compute_gradient(y, x, w)
gradient.shape

(30,)

In [5]:
w_init = np.random.rand(x.shape[1])
w, loss = least_squares_GD(y, x, w_init, max_iters=100, gamma=0.0000003)

Gradient Descent(0/99): loss=9314585.010996735		0.339944
Gradient Descent(1/99): loss=4894723.352659363		0.626876
Gradient Descent(2/99): loss=2590852.2618884495		0.342664
Gradient Descent(3/99): loss=1387373.5026232044		0.626976
Gradient Descent(4/99): loss=756649.92168339		0.342668
Gradient Descent(5/99): loss=424432.64680752525		0.62608
Gradient Descent(6/99): loss=248084.73937329336		0.342668
Gradient Descent(7/99): loss=153353.304449154		0.619288
Gradient Descent(8/99): loss=101534.3357602267		0.342668
Gradient Descent(9/99): loss=72416.29336609752		0.606804
Gradient Descent(10/99): loss=55416.65611938198		0.347908
Gradient Descent(11/99): loss=44973.5278868575		0.591972
Gradient Descent(12/99): loss=38147.791193514466		0.395108
Gradient Descent(13/99): loss=33374.15027100143		0.570916
Gradient Descent(14/99): loss=29809.63999582467		0.44698
Gradient Descent(15/99): loss=26993.095931255302		0.548068
Gradient Descent(16/99): loss=24666.655425712812		0.47704
Gradient Descent(17/99):

In [6]:
# w, loss = least_squares_SGD(y, x, w, 10, max_iters=100, gamma=0.0000001)

In [10]:
get_accuracy(x, y, w)

0.470496

In [8]:
y_test, x_test, ids_test = load_csv_data(data_path="datas/test.csv", sub_sample=False)
pred_y = predict_labels(w, x_test)


In [9]:
create_csv_submission(ids_test, pred_y, "datas/submission.csv")
print('Done !')


Done !
